In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
os.environ['LANGSMITH_PROJECT']="ai-agents-rags"
os.environ['LANGSMITH_TRACING']="true"

In [3]:
LANGSMITH_TRACING = True

### Find User Agent

In [4]:
import requests

response = requests.get('https://httpbin.org/user-agent')
print(response.json())

{'user-agent': 'python-requests/2.32.3'}


## Overview

In [5]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

## Loading, Splitting, Retrieval and Generation: from a webpage

### Indexing
https://python.langchain.com/docs/integrations/document_loaders/
![Indexing](./indexing.png)

In [6]:
# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

### Splitting
This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is ["\n\n", "\n", " ", ""]. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

In [7]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed https://python.langchain.com/docs/integrations/vectorstores/
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

### Generation
![Generation](./generation.png)

In [8]:
#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is the process of breaking down a complex task into smaller, manageable steps to facilitate planning and execution. Techniques like Chain of Thought (CoT) and Tree of Thoughts enhance this process by encouraging step-by-step reasoning and exploring multiple possibilities. It can be achieved through simple prompting, task-specific instructions, or human inputs, and may also involve external classical planners for long-horizon planning.'

In [9]:
rag_chain.invoke("""
How to optimize the LLM calls in a RAG setup, for example during a conversation - What measures to be taken to remember the past without stacking the converstion?
""")

"To optimize LLM calls in a RAG setup during conversations, consider implementing a long-term memory system using external vector stores for efficient retrieval of historical context without overloading the LLM's short-term memory. Additionally, utilize summarization techniques to condense past interactions, allowing the model to maintain relevant information while minimizing context length. This approach helps balance the need for historical awareness with the limitations of the LLM's context capacity."

In [10]:
rag_chain.invoke("""
How exactly the LLM - Vector store interaction happens during the conversation?
""")

'The interaction between the LLM and the vector store occurs through a retrieval model that surfaces relevant context based on recency, importance, and relevance to the current query. The LLM uses this retrieved information to inform its responses and generate outputs, while also leveraging both short-term and long-term memory for improved performance. This process allows the LLM to access a larger knowledge pool and adapt its behavior based on past experiences.'

In [11]:
rag_chain.invoke("""
Take the case openai chat completion api, we call the api with a specific question. 
Let us say during the 8th interaction of the conversation - context lost the first 5 interactions.
But the 8th interaction should refer the older data. What exactly going on here, Is the chat api makes a call to the vector store?
""")

'In the scenario described, if the chat API loses context from the first five interactions, it typically relies on its short-term memory, which is limited. However, if the 8th interaction needs to refer back to older data, it may utilize an external vector store to retrieve relevant information. This allows the model to access a larger knowledge pool beyond its immediate context.'

In [12]:
rag_chain.invoke("""
Assume the llm has no access to the vector store, then whats the significance of the vector store?
""")

"The vector store is significant because it provides access to a larger knowledge pool, allowing the LLM to retrieve information beyond its limited context capacity. This external memory capability enables the model to retain and recall information over extended periods, enhancing its performance in tasks requiring long-term memory. Without access to the vector store, the LLM's ability to utilize historical information and detailed instructions is severely restricted."

In [13]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

In [14]:

# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "o200k_base")

8

In [15]:

#https://python.langchain.com/docs/integrations/text_embedding/openai/
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)

1536

In [16]:
# https://platform.openai.com/docs/guides/embeddings#faq
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.8806915835035412
